In [2]:
import numpy as np
import math as math

## Задача 1

Контрольная работа состоит из пяти вопросов. На каждый вопрос приведено четыре варианта ответа, один из которых правильный. Составьте закон распределения числа правильных ответов при простом угадывании. Найдите `M(X)`, `D(X)`.

==================================================================================================================

Исходя из задачи, для начала нам нужно определить с какой вероятностью можно выбрать наугад:
1. 0 правильных вариантов из 5
2. 1 правильный вариант из 5
3. 2 правильных варианта из 5
4. 3 правильных варианта из 5
5. 4 правильных варианта из 5
6. 5 правильных вариантов из 5  

Очевидно что событие принимает вид "угадал/не угадал" ответ на вопрос. Так же очевидно что сумма вероятностей подобных исходов равна 1, потому что других исходов быть по умолчанию не может.....хотя можно контрольную прогулять в принципе))
Исходя из этого можно предположить, что вероятноть наступления события угадывания вопроса можно описать биномиальным распределением, где n=5, k=(от 0 до 5), p=1/4

In [23]:
def combinations(n, k):
    return math.factorial(n)/(math.factorial(k) * math.factorial(n - k))

In [24]:
def binomial_proba(n, k, p):
    return combinations(n, k) * (p ** k) * ((1 - p) ** (n - k))

In [34]:
# запишем возможные исходы в массив
success = np.arange(0, 6)
success

array([0, 1, 2, 3, 4, 5])

In [35]:
# посчитаем вероятности возможных исходов
success_proba = np.array([binomial_proba(5, k, 1/4) for k in range(0, 6)])
success_proba

array([0.23730469, 0.39550781, 0.26367188, 0.08789062, 0.01464844,
       0.00097656])

In [36]:
print(f"Сумма посчитанных вероятностей равна {success_proba.sum()}")

Сумма посчитанных вероятностей равна 1.0


Отлично, значит движемся в правильном направлении. Далее можно воспольоваться свойствами биномиального распределения или посчитать вручную. Сделаем и так и так чтобы проверить. Для начала воспользуемся свойствами

In [42]:
print(f"Математическое ожидание равно {5 * 1/4:.2f}")
print(f"Дисперсия равна {5 * 1/4 * (1 - 1/4):.2f}")

Математическое ожидание равно 1.25
Дисперсия равна 0.94


Посчитаем вручную матожидание

In [43]:
expected_value = np.sum(success * success_proba)
print(f"Математическое ожидание, посчитанное вруную, равно {expected_value:.2f}")

Математическое ожидание, посчитанное вруную, равно 1.25


Посчитаем дисперсию

In [48]:
variance = np.sum(((success - expected_value) ** 2 ) * success_proba)
print(f"Дисперсия, посчитанная вруную, равна {variance:.2f}")

Дисперсия, посчитанная вруную, равна 0.94


Таким образом если смотреть на закон распределения ответов, угаданных на удачу, то в среднем можно ориентироваться на один правильный ответ....поэтому это не самая лучшая стратегия для контрольной))

## Задача 2

Пользователь получает в среднем 10 писем со спамом на свой почтовый ящик за сутки. Найти число N, такое, что с вероятностью 0.95 пользователь получит не более N писем со спамом за текущий день.

=============================================================================================================

Исходя из задачи, нам нужно классифицировать письма на спам и не спам. Поскольку дано среднее число писем в день, то логично использовать распределение пуассона. В принципе можно было бы использовать и биномиальное распределение, но для этого нужно знать еще один параметр - количество получаемых писем всего за день, его не дано, поэтому будем использовать закон пуассона.     Закон распределения пуассона известен, и получается нам известна искомая вероятность, а сам параметр k -нет. В принципе ничего сложного нет, мы можем либо перебирать в бесконечном цикле параметр k до тех пор пока сумма вероятнотей предполагаемых писем не будет больше или равной 0.95

In [49]:
# Создадим фнкцию оценки вероятности распределения величины по закону Пуассона
def puasson_proba(lamb_da, k):
    return (lamb_da ** k) / math.factorial(k) * math.exp(-lamb_da)

In [71]:
n = 0
total_proba = 0
while True:
    proba = puasson_proba(10, n)
    total_proba += proba
    if total_proba >= 0.95:
        break
    n += 1

In [72]:
print(f"С вероятностью {total_proba:%} можно сказать что пользователь получит не более {n} писем за день")

С вероятностью 95.125960% можно сказать что пользователь получит не более 15 писем за день


Посчитаем вероятность для 14 писем

In [73]:
my_proba = 0
for i in range(0,15):
    my_proba += puasson_proba(10, i)

In [74]:
my_proba

0.9165415270653373

## Задача 3

Производятся выстрелы по мишени. Вероятность попадания в мишень при одном выстреле равна 0.01. Сколько выстрелов нужно сделать чтобы быть уверенным с вероятностью 0.9, что хотя бы 6 раз будет совершено попадание?

Подсказка: 1) "Вероятность попасть k раз при n выстрелах" - на какое распределение это похоже? 2) А если нам нужна вероятность P(X >= k), а не P(X = k)? 3) Здесь предстоит немножко покодить.

=========================================================================================================================

Итак по задаче кто-то куда-то стреляет. Соответсвенно событием является попал/не попал в мишень. Здесь необходимо использовать биномиальное распределение, поскольку событие может принимать только два исхода, и известен параметр требуемых успешных попаданий. Если бы был известен параметр сколько в среднем производится выстрелов за период, то тогда надо было бы использовать распределение пуассона, но задача бы стояла тогда скорее в другом ключе....из разряда определить вероятность попадания 6 раз при среднем количестве выстрелов N раз.  
  
Итак, закон распределения извествен, извествен параметр k, известен параметр p,неизвестен параметр n. В принципе задача сводиться практически к предыдушей. Нам нужно в цикле бегать до тех пор, пока сумма вероятностей исходов не будет больше или равна 0.9. Однако теперь нужно учесть следующее. Поскольку по умолчанию нам нужно сделать 6 попаданий, то как минимум нужно будет сделать 6 выстрелов. Таким образом цикл будет следующий:

In [75]:
n3 = 6
total_proba_3 = 0
while True:
    proba = binomial_proba(n3, 6, 0.01)
    total_proba_3 += proba
    if total_proba_3 >= 0.9:
        break
    n3 += 1

In [76]:
print(f"С вероятностью {total_proba_3:%} нужно сделать {n3} выстрелов, чтобы попасть 6 раз")

С вероятностью 91.851297% нужно сделать 230 выстрелов, чтобы попасть 6 раз
